### This code applies augmentation by raandoming deleting red box regions (used for random_shuffle and split_by_match).

In [1]:
import random
import os 
import cv2
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import ToPILImage
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim
from torchvision import transforms
from sklearn.model_selection import train_test_split
import shutil

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
def augment_image(image, rect_coords):
    draw = ImageDraw.Draw(image)
    num_boxes = len(rect_coords)

    # randomly decide how many boxes to mask (1 to num_boxes-1)
    num_to_mask = random.randint(1, num_boxes - 1)
    boxes_to_mask = random.sample(range(num_boxes), num_to_mask)

    for i, rect in enumerate(rect_coords):
        x1, y1 = min(rect[0], rect[2]), min(rect[1], rect[3])
        x2, y2 = max(rect[0], rect[2]), max(rect[1], rect[3])

        if i in boxes_to_mask:
            # Mask this region with black
            draw.rectangle([x1, y1, x2, y2], fill=(0, 0, 0))

    return image

In [ ]:
def augment_and_save(frames_dir, output_dir, rect_coords, augmentations_per_frame=3):
    os.makedirs(output_dir, exist_ok=True)
    for match_id in os.listdir(frames_dir):
        match_folder = os.path.join(frames_dir, match_id)
        if not os.path.isdir(match_folder):
            continue

        output_match_folder = os.path.join(output_dir, match_id)
        os.makedirs(output_match_folder, exist_ok=True)

        for frame in os.listdir(match_folder):
            if not frame.endswith('.jpg'):
                continue

            frame_path = os.path.join(match_folder, frame)
            image = Image.open(frame_path).convert("RGB")

            # save original
            original_path = os.path.join(output_match_folder, frame)
            image.save(original_path)

            # apply augmentations
            for i in range(augmentations_per_frame):
                augmented_image = augment_image(image.copy(), rect_coords)
                augmented_path = os.path.join(
                    output_match_folder, f"{os.path.splitext(frame)[0]}_augment_{i + 1}.jpg"
                )
                augmented_image.save(augmented_path)
                

In [ ]:
frames_dir = './data/frames'  
augmented_output_dir = './data/augmented_frames_no_mask'  

In [3]:
rect_coords = [
    (675, 10, 1242, 85),
    (1755, 5, 1916, 72),
    (292, 95, 2, 5),
    (1630, 793, 1915, 1074),
    (597, 846, 1335, 1077),
    (1828, 126, 1918, 648),
    (83, 132, 1, 655),
]

In [ ]:
augment_and_save(frames_dir, augmented_output_dir, rect_coords, augmentations_per_frame=3)